In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import os
os.chdir(r'C:\Users\半兵卫\Documents\006 项目\项目1 自动化建模')
from Auto_model.ExcelReader import excelReader, getModelVars
from Auto_model.GetDataSets import getDataSets
from Auto_model.CharWoe import charWoe
from Auto_model.ProcessMissExtreme import processMissExtreme
from Auto_model.Auto_Xgb import auto_xgb
from Auto_model.IV import Iv
from Auto_model.PSI import Psi
from Auto_model.BivarPlot import bivarPlot

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
uid, weight, dep = "bg_order_no_vouch", "weight", "bg_result_compensate"
keep_vars = [uid, weight,dep]  #不参与训练的列名，包括用户唯一标识、权重weight、样本标签三个信息

# 数据集处理

In [3]:
#设置weight参数表示权重，因为数据的正样本做过1/4的下采样，负样本取全部。
#训练的时候用不到，但是就是ks和psi会用得到，为了符合真实的业务场景
df6 = pd.read_csv(r'data\6m.csv',header=0)
df7 = pd.read_csv(r"data\7m.csv", header=0)
df8 = pd.read_csv(r"data\8m.csv", header=0)
df = pd.concat([df6,df7,df8])
df["weight"] = 4
df.loc[df[dep] == 1, weight] = 1
df = df.sample(50000)
df.index = range(df.shape[0])

offf = pd.read_csv(r"data\9m.csv", header=0)
offf["weight"] = 4
offf.loc[offf[dep] == 1, weight] = 1
offf = offf.sample(10000)
offf.index = range(offf.shape[0])

#切分训练集和测试集
devf = df[df.index % 10 < 6]
valf = df[df.index % 10 >= 6]
devf.to_csv(r"data\dev_raw_data.csv", index=None)
valf.to_csv(r"data\val_raw_data.csv", index=None)
offf.to_csv(r"data\off_raw_data.csv", index=None)

devf.shape
valf.shape
offf.shape

D:\python3\anconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
D:\python3\anconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,3,33,83,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(30000, 186)

(20000, 186)

(10000, 186)

# woe处理

In [6]:
#先将以前的mdel_trace删除,再做getmodelvars
getModelVars(path=r"data\off_raw_data.csv", dep=dep, weight=weight,
             outfile=r'report\model_trace.xlsx')
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

var_names: 145


In [7]:
# 删除"bg_order_no_vouch"
# 字符串woe处理
var_names = excelReader(file=r"report\model_trace.xlsx",   #读取model_trace中的字段名
                        sheet="vars", 
                        var_names=[]).getcols()  
print("var_names: %s" % len(var_names))
vars = [
     "hz_cargo_type_name",
     "hz_city_name",
     "hz_departure_name",
     "hz_destination_name",
     "hz_vehicle_type_name",
     "sj_30day_auth",
     "sj_30day_register",
     "sj_7day_auth",
     "sj_7day_register",
     "sj_card_name",
     "sj_city_name",
     "sj_etc_open",
     "sj_vehicle_type",
     "sj_vip_type",
     "sj_p_cargo_truck_type",
     "sj_plate_city_name",
     "sj_plate_number",
     "sj_plate_province_name"
]

var_names: 144


In [8]:
from Auto_model.CharWoe import charWoe
datasets = getDataSets(devp=r'data\dev_raw_data.csv',
                        valp=r'data\val_raw_data.csv',
                        offp=r'data\off_raw_data.csv',
                        usecols=var_names+[uid, weight, dep]+vars  #导入var_names,vars,uid,dep,weight
                      ).loading()  

print("已将三个数据集存入到字典中...")
#char_woe函数的逻辑如下：
#如果初始箱数超过100，返回变量名称 + contains too many different values...，这个特征建议做截断之后再做woe，这里直接跳过
#初始箱数不超过100，返回变量名称 + 箱的个数，并且计算每一箱的badrate，排序
#箱数超过10，就进行合并，相邻两箱的badrate做差值，最小的合并
#计算每一箱的样本占比，占比不超过0.05就需要合并
#如果某一箱不到0.05而且箱的数量大于4，就与第二箱合并，如果最后一箱不超过0.05就与倒数第二箱进行合并，
#如果是中间的箱，左右两边badrate相差小的进行合并
datasets = charWoe(datasets=datasets,                   
                   keep_vars=[uid, weight, dep], 
                   weight=weight, 
                   dep=dep, 
                   vars=vars).char_woe()

datasets.get("dev", "").to_csv(r"data\dev_raw_data_woe.csv", index=None)  #将字典中的三个数据集导出生成csv文件
datasets.get("val", "").to_csv(r"data\val_raw_data_woe.csv", index=None)
datasets.get("off", "").to_csv(r"data\off_raw_data_woe.csv", index=None)

已将三个数据集存入到字典中...
hz_cargo_type_name 该变量中，值的类型超过100个，建议先做截断处理...
hz_city_name 该变量中，值的类型超过100个，建议先做截断处理...
hz_departure_name 该变量中，值的类型超过100个，建议先做截断处理...
hz_destination_name 该变量中，值的类型超过100个，建议先做截断处理...
hz_vehicle_type_name 特征中取值的个数： 80
sj_30day_auth 特征中取值的个数： 6
sj_30day_register 特征中取值的个数： 4
sj_7day_auth 特征中取值的个数： 6
sj_7day_register 特征中取值的个数： 4
sj_card_name 特征中取值的个数： 14
sj_city_name 该变量中，值的类型超过100个，建议先做截断处理...
sj_etc_open 特征中取值的个数： 6
sj_vehicle_type 特征中取值的个数： 22
sj_vip_type 特征中取值的个数： 4
sj_p_cargo_truck_type 特征中取值的个数： 31
sj_plate_city_name 该变量中，值的类型超过100个，建议先做截断处理...
sj_plate_number 特征中取值的个数： 62
sj_plate_province_name 特征中取值的个数： 62


# 训练模型（使用特征重要性迭代删除变量）

In [12]:
#取列名，放入model_trace.xlsx中
#先将以前的mdel_trace删除,再做getmodelvars
getModelVars(path=r"data\off_raw_data_woe.csv", dep=dep, weight=weight,
             outfile=r'report\model_trace.xlsx')
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

var_names: 157


In [13]:
#删除model_trace文件中的标签
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
#将做过woe后的三个数据集重新导入字典中
datasets = getDataSets(devp=r'data\dev_raw_data_woe.csv',
                       valp=r'data\val_raw_data_woe.csv',
                       offp=r'data\off_raw_data_woe.csv',
                       usecols=var_names + [uid, weight, dep]).loading()

var_names: 156


In [15]:
# xgboost 算法训练
#生成Xgboost_train_tmp报表
#因为存在特征重要性的不一致性，比如我要删除特诊重要性小于20的特征，有个特征重要性分别是18和19,18是一定要删除的
#19不一定，因为删除18的特征之后，重新计算特征重要性，原本重要性为19的可能变成20了
#报表中返回每次删除10个特征重要性小于0.02的特征之后的，模型训练集、测试集和跨时间验证集的ks
#在测试集和跨时间验证集上的psi，以及删除的特征名称
params = {
    "max_depth": 2,
    "learning_rate": 0.09,
    "n_estimators": 120,
    "min_child_weight": 50,
    "subsample": 1,
    "scale_pos_weight": 1,
    "reg_lambda": 30
}
auto_xgb(datasets=datasets,
        uid=uid, 
        dep=dep, 
        weight=weight, 
        var_names=var_names, 
        params=params,
        max_del_var_nums=10).training(min_score=0.02 )

del var:  156 --> 146 ks:  {'devks': 0.3906537622399739, 'valks': 0.3585135129011843, 'offks': 0.369947581397818, 'valpsi': 0.0006469255304140798, 'offpsi': 0.013861970079836843} sj_day_auth,sj_day_register,sj_vehicle_length,sj_load,sj_cnt_ht_empty,sj_amt_ht_empty,sj_cnt_compensate,sj_cnt_compensate_suceess,sj_cnt_vouch_driver,sj_p_cargo_line_length
del var:  146 --> 136 ks:  {'devks': 0.3841446823237685, 'valks': 0.3527905617373363, 'offks': 0.35891807339680715, 'valpsi': 0.0005354408114000744, 'offpsi': 0.0040032202511867294} sj_p_cargo_truck_length,sj_p_cargo_truck_load,sj_day_fis_vouch_rate,sj_30_days_cnt_ht_empty,sj_30_days_amt_ht_empty,sj_30_days_cnt_compensate,sj_30_days_cnt_compensate_suceess,sj_30_days_cnt_compensate_rate,sj_30_days_cnt_compensate_suceess_rate,sj_60_days_cnt_ht_empty
del var:  136 --> 126 ks:  {'devks': 0.386157481271997, 'valks': 0.3547906493412338, 'offks': 0.3576545176947949, 'valpsi': 0.0005588516411555766, 'offpsi': 0.003973806020795707} sj_60_days_amt_ht

# 计算IV、psi和bivar

In [24]:
#step6: 计算变量IV
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
Iv(datapath=r'data\dev_raw_data_woe.csv', 
   dep=dep, 
   var_names=var_names, 
   outfile=r'report\IV.xlsx').sloveIv()

var_names: 26
0 sj_90_days_cnt_compensate_rate 8
1 sj_90_days_cnt_compensate_suceess_rate 8
2 sj_60_days_cnt_compensate 5
3 sj_60_days_cnt_compensate_rate 8
4 sj_90_days_cnt_compensate 6
5 hz_30_days_cnt_cargo_compensate 7
6 hz_60_days_cnt_compensate_rate 8
7 hz_90_days_cnt_compensate_rate 8
8 hz_90_days_cnt_compensate_suceess_rate 8
9 hz_avg_cargo_call_times_last30d_rate 8
10 hz_30_days_cnt_compensate_suceess_rate 8
11 sj_plate_province_name_woe 8
12 sj_age 8
13 si_fis_vouch_90day_rate 8
14 sj_view_30days_cnt 8
15 sj_p_cargo_truck_type_woe 5
16 hz_90_days_cnt_compensate_suceess 7
17 hz_60_days_cnt_compensate_suceess_rate 8
18 hz_60_days_cnt_cargo 8
19 hz_30_days_cnt_cargo_fis 8
20 sj_plate_number_woe 8
21 hz_60_days_cnt_vouch_consignor 8
22 hz_vehicle_type_name_woe 5
23 hz_90_days_cnt_cargo 8
24 sj_card_name_woe 5
25 hz_90_days_cnt_vouch_consignor 8


In [25]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
Psi(datasets=datasets, dep=dep, var_names=var_names, ivfile=r'report\IV.xlsx', outfile=r"report\PSI.xlsx").slove()

var_names: 26
0 sj_90_days_cnt_compensate_rate
1 sj_90_days_cnt_compensate_suceess_rate
2 sj_60_days_cnt_compensate
3 sj_60_days_cnt_compensate_rate
4 sj_90_days_cnt_compensate
5 hz_30_days_cnt_cargo_compensate
6 hz_60_days_cnt_compensate_rate
7 hz_90_days_cnt_compensate_rate
8 hz_90_days_cnt_compensate_suceess_rate
9 hz_avg_cargo_call_times_last30d_rate
10 hz_30_days_cnt_compensate_suceess_rate
11 sj_plate_province_name_woe
12 sj_age
13 si_fis_vouch_90day_rate
14 sj_view_30days_cnt
15 sj_p_cargo_truck_type_woe
16 hz_90_days_cnt_compensate_suceess
17 hz_60_days_cnt_compensate_suceess_rate
18 hz_60_days_cnt_cargo
19 hz_30_days_cnt_cargo_fis
20 sj_plate_number_woe
21 hz_60_days_cnt_vouch_consignor
22 hz_vehicle_type_name_woe
23 hz_90_days_cnt_cargo
24 sj_card_name_woe
25 hz_90_days_cnt_vouch_consignor


In [21]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))
bivarPlot(datasets=datasets, dep=dep, var_names=var_names, ivfile=r'report\IV.xlsx', outfile=r"report\Bivar.xlsx").plot()

var_names: 26
0 sj_90_days_cnt_compensate_rate
1 sj_90_days_cnt_compensate_suceess_rate
2 sj_60_days_cnt_compensate
3 sj_60_days_cnt_compensate_rate
4 sj_90_days_cnt_compensate
5 hz_30_days_cnt_cargo_compensate
6 hz_60_days_cnt_compensate_rate
7 hz_90_days_cnt_compensate_rate
8 hz_90_days_cnt_compensate_suceess_rate
9 hz_avg_cargo_call_times_last30d_rate
10 hz_30_days_cnt_compensate_suceess_rate
11 sj_plate_province_name_woe
12 sj_age
13 si_fis_vouch_90day_rate
14 sj_view_30days_cnt
15 sj_p_cargo_truck_type_woe
16 hz_90_days_cnt_compensate_suceess
17 hz_60_days_cnt_compensate_suceess_rate
18 hz_60_days_cnt_cargo
19 hz_30_days_cnt_cargo_fis
20 sj_plate_number_woe
21 hz_60_days_cnt_vouch_consignor
22 hz_vehicle_type_name_woe
23 hz_90_days_cnt_cargo
24 sj_card_name_woe
25 hz_90_days_cnt_vouch_consignor


In [26]:
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

var_names: 26


# 自动化调参

In [22]:
# step7: 参数自动化调整
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

auto_xgb(datasets=datasets, uid=uid, dep=dep, weight=weight, var_names=var_names,
         params=dict(), max_del_var_nums=0).auto_choose_params(target="weight")

var_names: 26
train_number: 0, devks: 0.4302872535420678, offks: 0.36418537597570844, params: {'max_depth': 5, 'learning_rate': 0.09, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 1, devks: 0.47163931828855815, offks: 0.3563576912134074, params: {'max_depth': 5, 'learning_rate': 0.14, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Good) train_number: 2, devks: 0.3822110616296378, offks: 0.35922778521069826, params: {'max_depth': 5, 'learning_rate': 0.04000000000000001, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 3, devks: 0.45521180169089986, offks: 0.3681416630643709, params: {'max_depth': 6, 'learning_rate': 0.09000000000000001, 'n_estimators': 120, 'min_child_weight': 50, 'subsample': 1, 'scale_pos_weight': 1, 'reg_lambda': 21}
(Bad) train_number: 4, devks: 0.41972023257862556

In [21]:
params =  {'max_depth': 5,
           'learning_rate': 0.09000000000000001, 
           'n_estimators': 120,
           'min_child_weight': 50,
           'subsample': 1.0, 
           'scale_pos_weight': 1, 
           'reg_lambda': 21}

# 递归特征删除（使用ks）

In [23]:
# step8: 变量自动化调整(是否可以删除一些无用的变量)
auto_xgb(datasets=datasets, uid=uid, dep=dep, weight=weight,
        var_names=var_names, params=params, max_del_var_nums=1).auto_delete_vars()

train_number: 0, offks: 0.34366948314855417
(Good) train_n: 1, offks: 0.3450191605058549 by vars: hz_30_days_cnt_compensate_rate
(Good) train_n: 2, offks: 0.35257476811793176 by vars: sj_90_days_cnt_compensate
(Good) train_n: 3, offks: 0.3529600208501881 by vars: sj_vehicle_type_woe
(Bad) train_n: 4, offks: 0.34913128094465 by vars: 22
(Bad) train_n: 5, offks: 0.3376282549007896 by vars: 22
(Bad) train_n: 6, offks: 0.33256127651553646 by vars: 22
(Good) train_n: 7, offks: 0.3547774312226913 by vars: sj_90_days_cnt_compensate_suceess
(Bad) train_n: 8, offks: 0.34472362770520465 by vars: 21
(Bad) train_n: 9, offks: 0.34768464400708343 by vars: 21
(Bad) train_n: 10, offks: 0.34524203826102606 by vars: 21
(Bad) train_n: 11, offks: 0.3484189512282969 by vars: 21
(Bad) train_n: 12, offks: 0.3461022636571447 by vars: 21
(Bad) train_n: 13, offks: 0.3513049854547701 by vars: 21
(Bad) train_n: 14, offks: 0.337514230434397 by vars: 21
(Bad) train_n: 15, offks: 0.34643425325997496 by vars: 21
(Bad

In [24]:
del_list = ['hz_30_days_cnt_compensate_rate',
            'sj_90_days_cnt_compensate',
            'sj_vehicle_type_woe',
            'sj_90_days_cnt_compensate_suceess']

for i in del_list:
    var_names.remove(i)
print("var_names: %s" % len(var_names))

var_names_end = {'变量': var_names}
pd.DataFrame(var_names_end).to_csv(r"report\var_names_end.csv", index=None)

var_names: 21


# 重新拟合模型并且生成报告

In [26]:
#生成报告
var_names = excelReader(file=r"report\model_trace.xlsx", sheet="vars", var_names=[]).getcols()
print("var_names: %s" % len(var_names))

params =  {'max_depth': 5,
           'learning_rate': 0.09000000000000001, 
           'n_estimators': 120,
           'min_child_weight': 50,
           'subsample': 1.0, 
           'scale_pos_weight': 1, 
           'reg_lambda': 21}

datasets = getDataSets(devp=r'data\dev_raw_data_woe_trt.csv',
                       valp=r'data\val_raw_data_woe_trt.csv',
                       offp=r'data\off_raw_data_woe_trt.csv',
                       usecols=var_names + [uid, weight, dep]).loading()

var_names: 21


In [33]:
auto_xgb(datasets=datasets,
        uid=uid, 
        dep=dep, 
        weight=weight, 
        var_names=var_names, 
        params=params,
        max_del_var_nums=0).training(min_score=0.0001, 
                             modelfile=r"report\model_report.pmml", 
                             output_scores=[])

del var:  21 --> 21 ks:  {'devks': 0.4101446942814385, 'valks': 0.3210244889270239, 'offks': 0.35465151304778614, 'valpsi': 0.0003588705121370242, 'offpsi': 0.011683389741147143} 
[431, 137, 103, 75, 65, 46, 69, 46, 35, 42, 25, 24, 23, 25, 27, 36, 13, 12, 8, 1]
dev ks: 0.4066353315185689
[201, 78, 55, 48, 47, 33, 30, 30, 29, 30, 21, 34, 19, 21, 19, 16, 15, 12, 11, 8]
val ks: 0.3195558905909685
[106, 38, 34, 27, 37, 12, 22, 17, 10, 16, 15, 13, 9, 9, 9, 8, 5, 8, 5, 3]
off ks: 0.3531273084460068


Exception ignored in: <function excelWriter.__del__ at 0x00000163777CB708>
Traceback (most recent call last):
  File "C:\Users\半兵卫\Documents\006 项目\项目1 自动化建模\Auto_model\ExcelWriter.py", line 35, in __del__
    self.workbook.close()
  File "D:\python3\anconda\lib\site-packages\xlsxwriter\workbook.py", line 318, in close
    raise FileCreateError(e)
xlsxwriter.exceptions.FileCreateError: [Errno 13] Permission denied: 'report\\Xgboost_train_tmp.xlsx'


In [34]:
import joblib
model = joblib.load( 'model.pkl') 
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.09000000000000001, max_delta_step=0, max_depth=5,
              min_child_weight=50, missing=None, monotone_constraints='()',
              n_estimators=120, n_jobs=10, nthread=10, num_parallel_tree=1,
              random_state=7, reg_alpha=0, reg_lambda=21, scale_pos_weight=1,
              subsample=1.0, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [35]:
#打分code
model.predict_proba(datasets['off'][var_names])[:,1]

array([0.31694534, 0.10454357, 0.02220125, ..., 0.02923585, 0.01579922,
       0.02337986], dtype=float32)

In [40]:
#生成报告
auto_xgb(datasets=datasets, uid=uid, dep=dep, weight=weight,
        var_names=var_names, params=params, max_del_var_nums=0).plotKS(model, bins=20)

[431, 137, 103, 75, 65, 46, 69, 46, 35, 42, 25, 24, 23, 25, 27, 36, 13, 12, 8, 1]
dev ks: 0.4066353315185689
[201, 78, 55, 48, 47, 33, 30, 30, 29, 30, 21, 34, 19, 21, 19, 16, 15, 12, 11, 8]
val ks: 0.3195558905909685
[106, 38, 34, 27, 37, 12, 22, 17, 10, 16, 15, 13, 9, 9, 9, 8, 5, 8, 5, 3]
off ks: 0.3531273084460068
